<a href="https://colab.research.google.com/github/osun24/nsclc-adj-chemo/blob/main/TorchSurv_DeepSurv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install torchsurv scikit-survival

# Import required packages
import os
import time
import datetime
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sksurv.metrics import concordance_index_censored

# (Optional) Mount Google Drive if you plan to load/save files there
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
import math
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsurv.loss.cox import neg_partial_log_likelihood
from sksurv.metrics import concordance_index_censored
import warnings
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import copy
import random

warnings.filterwarnings("ignore", message="Ties in event time detected; using efron's method to handle ties.")
torch.manual_seed(0); np.random.seed(0); random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ---------------- utils ----------------
class Tee:
    def __init__(self, *files): self.files = files
    def write(self, data):
        for f in self.files: f.write(data)
    def flush(self):
        for f in self.files: f.flush()

def loguniform(rng, lo, hi):
    return float(np.exp(rng.uniform(np.log(lo), np.log(hi))))

def sample_hparams(rng):
    hidden_options = [[8], [16], [32], [16, 8], [32, 16]]
    layers = hidden_options[rng.integers(len(hidden_options))]
    dropout = float(rng.uniform(0.4, 0.7))             # narrower, slightly higher
    lr = loguniform(rng, 2e-5, 2e-4)                   # lower LR band
    wd = loguniform(rng, 2e-3, 5e-1)                   # stronger L2 via AdamW
    l1 = loguniform(rng, 5e-5, 5e-3)                   # modest L1
    return {'layers': layers, 'dropout': dropout, 'lr': lr, 'wd': wd, 'l1': l1}

# ---------------- model & data ----------------
class DeepSurvMLP(nn.Module):
    def __init__(self, in_features, hidden_layers, dropout=0.0, activation=nn.ReLU()):
        super().__init__()
        layers, d = [], in_features
        for units in hidden_layers:
            layers += [nn.Linear(d, units), activation]
            if dropout > 0: layers.append(nn.Dropout(dropout))
            d = units
        layers.append(nn.Linear(d, 1))
        self.model = nn.Sequential(*layers)
    def forward(self, x): return self.model(x)

class SurvivalDataset(Dataset):
    def __init__(self, features, time_vals, events):
        self.x = torch.tensor(features, dtype=torch.float32)
        self.time = torch.tensor(time_vals, dtype=torch.float32)
        self.event = torch.tensor(events, dtype=torch.bool)
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx], self.time[idx], self.event[idx]

def l1_penalty(model):
    return sum(p.abs().sum() for n,p in model.named_parameters()
               if p.requires_grad and p.dim() > 1)

def train_one_epoch(model, optimizer, dataloader, device, l1_lambda=0.0):
    model.train()
    for x, t, e in dataloader:
        if e.sum().item() == 0:  # skip non-informative batch
            continue
        x, t, e = x.to(device), t.to(device), e.to(device)
        optimizer.zero_grad()
        out = torch.clamp(model(x), -20, 20)
        loss = neg_partial_log_likelihood(out, e, t, reduction='mean')
        if l1_lambda > 0: loss = loss + l1_lambda * l1_penalty(model)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()

def evaluate_ci(model, dataloader, device):
    model.eval()
    preds, times, events = [], [], []
    with torch.no_grad():
        for x, t, e in dataloader:
            x = x.to(device)
            y = torch.clamp(model(x), -20, 20)
            preds.append(y.cpu().numpy().ravel())
            times.append(t.numpy()); events.append(e.numpy())
    preds = np.concatenate(preds)
    if np.isnan(preds).any():
        print("Warning: NaN predictions detected, returning -inf for concordance index")
        return -np.inf
    times = np.concatenate(times); events = np.concatenate(events)
    return concordance_index_censored(events.astype(bool), times, preds)[0]

def count_params(in_dim, hidden_layers):
    params, d = 0, in_dim
    for h in hidden_layers:
        params += d*h + h  # weights + bias
        d = h
    params += d*1 + 1     # final layer
    return params

def collect_val_arrays(model, dataloader, device):
    model.eval()
    preds, times, events = [], [], []
    with torch.no_grad():
        for x, t, e in dataloader:
            x = x.to(device)
            y = torch.clamp(model(x), -20, 20).squeeze(1)
            preds.append(y.cpu().numpy())
            times.append(t.numpy()); events.append(e.numpy())
    return np.concatenate(preds), np.concatenate(times), np.concatenate(events)

def bootstrap_se_cindex(preds, times, events, B=200, seed=123):
    rng = np.random.default_rng(seed)
    n = len(times)
    vals = []
    for _ in range(B):
        idx = rng.integers(0, n, n)
        vals.append(concordance_index_censored(events[idx].astype(bool), times[idx], preds[idx])[0])
    return np.std(vals, ddof=1)

# ---------------- random search with successive halving ----------------
def main():
    # logging
    original_stdout = sys.stdout
    log_path = "/content/drive/MyDrive/deepsurv_8-28-25_training_log.txt"
    with open(log_path, "w") as log_file:
        sys.stdout = Tee(original_stdout, log_file)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        current_date = datetime.datetime.now().strftime("%Y%m%d")
        output_dir = "/content/drive/MyDrive/deepsurv_results"
        os.makedirs(output_dir, exist_ok=True)

        # ----- load & prep data -----
        train_df = pd.read_csv("/content/drive/MyDrive/affyTrain.csv")
        valid_df = pd.read_csv("/content/drive/MyDrive/affyValidation.csv")
        for df in (train_df, valid_df):
            if 'Adjuvant Chemo' in df.columns:
                df['Adjuvant Chemo'] = df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})
        binary_columns = ['Adjuvant Chemo','IS_MALE']
        for col in binary_columns:
            if col in train_df.columns: train_df[col] = train_df[col].astype(int)
            if col in valid_df.columns: valid_df[col] = valid_df[col].astype(int)

        survival_cols = ['OS_STATUS','OS_MONTHS']
        feature_cols = [c for c in train_df.columns if c not in survival_cols]

        X_train = train_df[feature_cols].values.astype(np.float32)
        X_valid = valid_df[feature_cols].values.astype(np.float32)
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train).astype(np.float32)
        X_valid = scaler.transform(X_valid).astype(np.float32)

        y_train_time = train_df['OS_MONTHS'].values.astype(np.float32)
        y_train_event = train_df['OS_STATUS'].values.astype(np.float32)
        y_valid_time = valid_df['OS_MONTHS'].values.astype(np.float32)
        y_valid_event = valid_df['OS_STATUS'].values.astype(np.float32)

        train_ds = SurvivalDataset(X_train, y_train_time, y_train_event)
        valid_ds = SurvivalDataset(X_valid, y_valid_time, y_valid_event)

        batch_size = 32
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  drop_last=True)
        train_eval_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
        valid_loader      = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

        # ----- search budget & rungs -----
        rng = np.random.default_rng(42)
        num_trials = 80                     # budget (adjust as needed)
        rungs = [8, 24, 48, 96]                # epochs at which we prune
        eta = 3                            # keep top 1/eta each rung
        patience, min_delta = 12, 1e-4      # early stopping inside trials
        PRINT_EVERY = 1

        # ----- initialize trials -----
        trials = []
        for tid in range(num_trials):
            hp = sample_hparams(rng)
            model = DeepSurvMLP(X_train.shape[1], hp['layers'], dropout=hp['dropout']).to(device)
            optimizer = optim.AdamW(model.parameters(), lr=hp['lr'], weight_decay=hp['wd'])
            trials.append({
                'id': tid, 'hp': hp, 'model': model, 'opt': optimizer,
                'best_ci': -np.inf, 'best_state': copy.deepcopy(model.state_dict()),
                'hist_train_ci': [], 'hist_val_ci': [], 'epochs_done': 0,
                'alive': True, 'no_improve': 0, 'last_val_ci': -np.inf
            })

        # ----- successive halving loop -----
        for rung_idx, rung_ep in enumerate(rungs, start=1):
            print(f"\n=== Rung {rung_idx}/{len(rungs)} → target {rung_ep} epochs ===")
            # train all alive trials up to rung_ep (with early stopping)
            for tr in trials:
                if not tr['alive']: continue
                target = rung_ep
                while tr['epochs_done'] < target:
                    train_one_epoch(tr['model'], tr['opt'], train_loader, device, l1_lambda=tr['hp']['l1'])
                    tr['epochs_done'] += 1
                    # diagnostics each epoch
                    tr_ci = evaluate_ci(tr['model'], train_eval_loader, device)
                    va_ci = evaluate_ci(tr['model'], valid_loader, device)
                    tr['hist_train_ci'].append(tr_ci)
                    tr['hist_val_ci'].append(va_ci)

                    if tr['epochs_done'] % PRINT_EVERY == 0:
                      print(f"[Trial {tr['id']:02d} | Rung {rung_idx}/{len(rungs)} | "
                            f"Epoch {tr['epochs_done']:3d}] Train CI {tr_ci:.4f} | "
                            f"Val CI {va_ci:.4f} | Best {tr['best_ci']:.4f}")

                    # track best
                    if va_ci > tr['best_ci'] + min_delta:
                        tr['best_ci'] = va_ci
                        tr['best_state'] = copy.deepcopy(tr['model'].state_dict())
                        tr['no_improve'] = 0
                    else:
                        tr['no_improve'] += 1
                        if tr['no_improve'] >= patience:
                            print(f"Trial {tr['id']} early-stopped at epoch {tr['epochs_done']} (best Val CI={tr['best_ci']:.4f})")
                            break

            # prune: keep top 1/eta among alive
            alive = [tr for tr in trials if tr['alive']]
            alive.sort(key=lambda z: z['best_ci'], reverse=True)
            keep_n = max(1, math.ceil(len(alive) / eta))
            survivors = set(tr['id'] for tr in alive[:keep_n])

            print(f"Alive before prune: {len(alive)}; keeping top {keep_n}")
            for tr in alive:
                if tr['id'] not in survivors:
                    tr['alive'] = False
                    # free memory
                    del tr['model']; tr['model'] = None
                    tr['opt'] = None
                    if torch.cuda.is_available(): torch.cuda.empty_cache()

        # ----- 1-SE model selection -----
        raw_best = max(trials, key=lambda z: z['best_ci'])

        tmp_model = DeepSurvMLP(X_train.shape[1], raw_best['hp']['layers'],
                                dropout=raw_best['hp']['dropout']).to(device)
        tmp_model.load_state_dict(raw_best['best_state'])
        val_preds, val_times, val_events = collect_val_arrays(tmp_model, valid_loader, device)
        se_best = bootstrap_se_cindex(val_preds, val_times, val_events, B=200, seed=123)
        threshold = raw_best['best_ci'] - se_best

        candidates = [tr for tr in trials if tr['best_state'] is not None and tr['best_ci'] >= threshold]

        def simplicity_score(tr):
            # prefer fewer params; if tie, prefer larger regularization & dropout
            return (count_params(X_train.shape[1], tr['hp']['layers']),
                    -(tr['hp']['dropout']),
                    -(tr['hp']['wd'] + tr['hp']['l1']))

        selected = min(candidates, key=simplicity_score)

        best_trial = selected
        best_hp = best_trial['hp']
        print("\n[1-SE] Raw best CI: {:.4f}, SE: {:.4f}, threshold: {:.4f}".format(
              raw_best['best_ci'], se_best, threshold))
        print("[1-SE] Selected Hyperparameters (simplest within 1-SE):")
        print(best_hp)
        print("Selected Validation CI:", best_trial['best_ci'])

        # ----- save plots per trial -----
        out_date = current_date
        for tr in trials:
            cfg = f"trial{tr['id']}_layers-{'-'.join(map(str, tr['hp']['layers']))}_drop{tr['hp']['dropout']:.2f}_lr{tr['hp']['lr']:.2e}_wd{tr['hp']['wd']:.2e}_l1{tr['hp']['l1']:.2e}"
            epochs = range(1, len(tr['hist_train_ci'])+1)
            plt.figure()
            plt.plot(epochs, tr['hist_train_ci'], label='Train CI')
            plt.plot(epochs, tr['hist_val_ci'], label='Val CI')
            plt.xlabel('Epoch'); plt.ylabel('Concordance Index'); plt.legend(); plt.grid(True, alpha=0.3)
            plt.title(cfg)
            plt.ylim(0.4, 1.0)
            plot_path = os.path.join(output_dir, f"{out_date}_ci_{cfg}.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight'); plt.close()
            print(f"Saved CI plot to {plot_path}")

        # ----- save results table -----
        results = []
        for tr in trials:
            row = {'trial_id': tr['id'], 'val_ci': tr['best_ci'], 'epochs_trained': len(tr['hist_val_ci']),
                   'alive_final': tr['alive']}
            row.update({ 'layers': '-'.join(map(str, tr['hp']['layers'])),
                         'dropout': tr['hp']['dropout'], 'lr': tr['hp']['lr'],
                         'weight_decay(L2)': tr['hp']['wd'], 'l1_lambda': tr['hp']['l1']})
            results.append(row)
        df = pd.DataFrame(results).sort_values('val_ci', ascending=False)
        csv_path = os.path.join(output_dir, f"{out_date}_deepsurv_randomSH_results.csv")
        df.to_csv(csv_path, index=False)
        print(f"Hyperparameter search results saved to {csv_path}")

        # ----- save best model -----
        best_model_path = os.path.join(output_dir, f"{out_date}_best_deepsurv_model.pth")
        torch.save(best_trial['best_state'], best_model_path)
        print(f"Best model saved to {best_model_path}")

        # ----- test evaluation -----
        test_df = pd.read_csv("/content/drive/MyDrive/affyTest.csv")
        if 'Adjuvant Chemo' in test_df.columns:
            test_df['Adjuvant Chemo'] = test_df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})
        for col in binary_columns:
            if col in test_df.columns: test_df[col] = test_df[col].astype(int)
        X_test = scaler.transform(test_df[feature_cols].values.astype(np.float32)).astype(np.float32)
        y_test_time = test_df['OS_MONTHS'].values.astype(np.float32)
        y_test_event = test_df['OS_STATUS'].values.astype(np.float32)

        test_loader = DataLoader(SurvivalDataset(X_test, y_test_time, y_test_event),
                                 batch_size=batch_size, shuffle=False)

        # rebuild & load best
        final_model = DeepSurvMLP(X_train.shape[1], best_hp['layers'], dropout=best_hp['dropout']).to(device)
        final_model.load_state_dict(torch.load(best_model_path, map_location=device))
        test_ci = evaluate_ci(final_model, test_loader, device)
        print(f"Test CI: {test_ci:.4f}")

        sys.stdout.flush()

    sys.stdout = original_stdout
    print("Training completed. Check your log file at:", log_path)

if __name__ == "__main__":
    main()

/tmp/ipython-input-3295702232.py:145: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Adjuvant Chemo'] = df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})



=== Rung 1/4 → target 8 epochs ===
[Trial 00 | Rung 1/4 | Epoch   1] Train CI 0.6955 | Val CI 0.5622 | Best -inf
[Trial 00 | Rung 1/4 | Epoch   2] Train CI 0.7655 | Val CI 0.5921 | Best 0.5622
[Trial 00 | Rung 1/4 | Epoch   3] Train CI 0.8029 | Val CI 0.6285 | Best 0.5921
[Trial 00 | Rung 1/4 | Epoch   4] Train CI 0.8350 | Val CI 0.6225 | Best 0.6285
[Trial 00 | Rung 1/4 | Epoch   5] Train CI 0.8495 | Val CI 0.6243 | Best 0.6285
[Trial 00 | Rung 1/4 | Epoch   6] Train CI 0.8713 | Val CI 0.6256 | Best 0.6285
[Trial 00 | Rung 1/4 | Epoch   7] Train CI 0.8788 | Val CI 0.6494 | Best 0.6285
[Trial 00 | Rung 1/4 | Epoch   8] Train CI 0.8979 | Val CI 0.6210 | Best 0.6494
[Trial 01 | Rung 1/4 | Epoch   1] Train CI 0.6131 | Val CI 0.5905 | Best -inf
[Trial 01 | Rung 1/4 | Epoch   2] Train CI 0.6578 | Val CI 0.6106 | Best 0.5905
[Trial 01 | Rung 1/4 | Epoch   3] Train CI 0.6653 | Val CI 0.6018 | Best 0.6106
[Trial 01 | Rung 1/4 | Epoch   4] Train CI 0.6599 | Val CI 0.6329 | Best 0.6106
[Trial 0

/tmp/ipython-input-3295702232.py:306: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['Adjuvant Chemo'] = test_df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})


Test CI: 0.6206
Training completed. Check your log file at: /content/drive/MyDrive/deepsurv_8-28-25_training_log.txt


In [1]:
import os
import sys
import math
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsurv.loss.cox import neg_partial_log_likelihood
from sksurv.metrics import concordance_index_censored
import warnings
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import copy
import random

warnings.filterwarnings("ignore", message="Ties in event time detected; using efron's method to handle ties.")
torch.manual_seed(0); np.random.seed(0); random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ---------------- utils ----------------
class Tee:
    def __init__(self, *files): self.files = files
    def write(self, data):
        for f in self.files: f.write(data)
    def flush(self):
        for f in self.files: f.flush()

def loguniform(rng, lo, hi):
    return float(np.exp(rng.uniform(np.log(lo), np.log(hi))))

def sample_hparams(rng):
    hidden_options = [[16], [32], [16, 8], [32, 16]]
    layers = hidden_options[rng.integers(len(hidden_options))]
    dropout = float(rng.uniform(0.4, 0.7))             # narrower, slightly higher
    lr = loguniform(rng, 2e-5, 2e-4)                   # lower LR band
    wd = loguniform(rng, 2e-3, 2e-2)                   # stronger L2 via AdamW
    l1 = loguniform(rng, 5e-6, 5e-5)                   # modest L1
    return {'layers': layers, 'dropout': dropout, 'lr': lr, 'wd': wd, 'l1': l1}

# ---------------- model & data ----------------
class DeepSurvMLP(nn.Module):
    def __init__(self, in_features, hidden_layers, dropout=0.0, activation=nn.ReLU()):
        super().__init__()
        layers, d = [], in_features
        for units in hidden_layers:
            layers += [nn.Linear(d, units), activation]
            if dropout > 0: layers.append(nn.Dropout(dropout))
            d = units
        layers.append(nn.Linear(d, 1))
        self.model = nn.Sequential(*layers)
    def forward(self, x): return self.model(x)

class SurvivalDataset(Dataset):
    def __init__(self, features, time_vals, events):
        self.x = torch.tensor(features, dtype=torch.float32)
        self.time = torch.tensor(time_vals, dtype=torch.float32)
        self.event = torch.tensor(events, dtype=torch.bool)
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx], self.time[idx], self.event[idx]

def l1_penalty(model):
    return sum(p.abs().sum() for n,p in model.named_parameters()
               if p.requires_grad and p.dim() > 1)

def train_one_epoch(model, optimizer, dataloader, device, l1_lambda=0.0):
    model.train()
    for x, t, e in dataloader:
        if e.sum().item() == 0:  # skip non-informative batch
            continue
        x, t, e = x.to(device), t.to(device), e.to(device)
        optimizer.zero_grad()
        out = torch.clamp(model(x), -20, 20)
        loss = neg_partial_log_likelihood(out, e, t, reduction='mean')
        if l1_lambda > 0: loss = loss + l1_lambda * l1_penalty(model)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()

def evaluate_ci(model, dataloader, device):
    model.eval()
    preds, times, events = [], [], []
    with torch.no_grad():
        for x, t, e in dataloader:
            x = x.to(device)
            y = torch.clamp(model(x), -20, 20)
            preds.append(y.cpu().numpy().ravel())
            times.append(t.numpy()); events.append(e.numpy())
    preds = np.concatenate(preds)
    if np.isnan(preds).any():
        print("Warning: NaN predictions detected, returning -inf for concordance index")
        return -np.inf
    times = np.concatenate(times); events = np.concatenate(events)
    return concordance_index_censored(events.astype(bool), times, preds)[0]

def evaluate_ci_ensemble(models, dataloader, device):
    for m in models: m.eval()
    preds, times, events = [], [], []
    with torch.no_grad():
        for x, t, e in dataloader:
            x = x.to(device)
            # average risk across models
            ys = [torch.clamp(m(x), -20, 20).squeeze(1) for m in models]
            y = torch.stack(ys, dim=0).mean(0)
            preds.append(y.cpu().numpy().ravel())
            times.append(t.numpy()); events.append(e.numpy())
    preds = np.concatenate(preds)
    if np.isnan(preds).any():
        print("Warning: NaN predictions detected, returning -inf for concordance index (ensemble)")
        return -np.inf
    times = np.concatenate(times); events = np.concatenate(events)
    return concordance_index_censored(events.astype(bool), times, preds)[0]

# ---------------- random search with successive halving ----------------
def main():
    # logging
    original_stdout = sys.stdout
    log_path = "/content/drive/MyDrive/deepsurv_8-28-25_training_log.txt"
    with open(log_path, "w") as log_file:
        sys.stdout = Tee(original_stdout, log_file)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        current_date = datetime.datetime.now().strftime("%Y%m%d")
        output_dir = "/content/drive/MyDrive/deepsurv_results"
        os.makedirs(output_dir, exist_ok=True)

        # ----- load & prep data -----
        train_df = pd.read_csv("/content/drive/MyDrive/affyTrain.csv")
        valid_df = pd.read_csv("/content/drive/MyDrive/affyValidation.csv")
        for df in (train_df, valid_df):
            if 'Adjuvant Chemo' in df.columns:
                df['Adjuvant Chemo'] = df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})
        binary_columns = ['Adjuvant Chemo','IS_MALE']
        for col in binary_columns:
            if col in train_df.columns: train_df[col] = train_df[col].astype(int)
            if col in valid_df.columns: valid_df[col] = valid_df[col].astype(int)

        survival_cols = ['OS_STATUS','OS_MONTHS']
        feature_cols_all = [c for c in train_df.columns if c not in survival_cols]

        K = 1000  # try 500/1000/1500 in separate runs
        X_train_full = train_df[feature_cols_all].values.astype(np.float32)
        X_valid_full = valid_df[feature_cols_all].values.astype(np.float32)

        var = X_train_full.var(axis=0)                 # <-- raw variance, not scaled
        topk_idx = np.argsort(var)[-K:]
        selected_feature_cols = [feature_cols_all[i] for i in topk_idx]

        # scale ONLY the selected features
        scaler = StandardScaler().fit(X_train_full[:, topk_idx])
        X_train = scaler.transform(X_train_full[:, topk_idx]).astype(np.float32)
        X_valid = scaler.transform(X_valid_full[:, topk_idx]).astype(np.float32)

        y_train_time = train_df['OS_MONTHS'].values.astype(np.float32)
        y_train_event = train_df['OS_STATUS'].values.astype(np.float32)
        y_valid_time = valid_df['OS_MONTHS'].values.astype(np.float32)
        y_valid_event = valid_df['OS_STATUS'].values.astype(np.float32)

        train_ds = SurvivalDataset(X_train, y_train_time, y_train_event)
        valid_ds = SurvivalDataset(X_valid, y_valid_time, y_valid_event)

        batch_size = 32
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  drop_last=True)
        train_eval_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
        valid_loader      = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

        # ----- search budget & rungs -----
        rng = np.random.default_rng(42)
        num_trials = 80
        rungs = [8, 24, 48, 96]
        eta = 3
        patience, min_delta = 12, 1e-4
        PRINT_EVERY = 1
        RETRAIN_TOP_N = 5
        MAX_FINAL_EPOCHS = 160
        FINAL_PATIENCE = 15

        # ----- initialize trials -----
        trials = []
        for tid in range(num_trials):
            hp = sample_hparams(rng)
            model = DeepSurvMLP(X_train.shape[1], hp['layers'], dropout=hp['dropout']).to(device)
            optimizer = optim.AdamW(model.parameters(), lr=hp['lr'], weight_decay=hp['wd'])
            trials.append({
                'id': tid, 'hp': hp, 'model': model, 'opt': optimizer,
                'best_ci': -np.inf, 'best_state': copy.deepcopy(model.state_dict()),
                'hist_train_ci': [], 'hist_val_ci': [], 'epochs_done': 0,
                'alive': True, 'no_improve': 0
            })

        # ----- successive halving loop -----
        for rung_idx, rung_ep in enumerate(rungs, start=1):
            print(f"\n=== Rung {rung_idx}/{len(rungs)} → target {rung_ep} epochs ===")
            # train all alive trials up to rung_ep (with early stopping)
            for tr in trials:
                if not tr['alive']: continue
                target = rung_ep
                while tr['epochs_done'] < target:
                    train_one_epoch(tr['model'], tr['opt'], train_loader, device, l1_lambda=tr['hp']['l1'])
                    tr['epochs_done'] += 1
                    # diagnostics each epoch
                    tr_ci = evaluate_ci(tr['model'], train_eval_loader, device)
                    va_ci = evaluate_ci(tr['model'], valid_loader, device)
                    tr['hist_train_ci'].append(tr_ci)
                    tr['hist_val_ci'].append(va_ci)

                    if tr['epochs_done'] % PRINT_EVERY == 0:
                        print(f"[Trial {tr['id']:02d} | Rung {rung_idx}/{len(rungs)} | "
                              f"Epoch {tr['epochs_done']:3d}] Train CI {tr_ci:.4f} | "
                              f"Val CI {va_ci:.4f} | Best {tr['best_ci']:.4f}")

                    # track best
                    if va_ci > tr['best_ci'] + min_delta:
                        tr['best_ci'] = va_ci
                        tr['best_state'] = copy.deepcopy(tr['model'].state_dict())
                        tr['no_improve'] = 0
                    else:
                        tr['no_improve'] += 1
                        if tr['no_improve'] >= patience:
                            print(f"Trial {tr['id']} early-stopped at epoch {tr['epochs_done']} (best Val CI={tr['best_ci']:.4f})")
                            break

            # prune: keep top 1/eta among alive
            alive = [tr for tr in trials if tr['alive']]
            alive.sort(key=lambda z: z['best_ci'], reverse=True)
            keep_n = max(1, math.ceil(len(alive) / eta))
            survivors = set(tr['id'] for tr in alive[:keep_n])

            print(f"Alive before prune: {len(alive)}; keeping top {keep_n}")
            for tr in alive:
                if tr['id'] not in survivors:
                    tr['alive'] = False
                    # free memory
                    del tr['model']; tr['model'] = None
                    tr['opt'] = None
                    if torch.cuda.is_available(): torch.cuda.empty_cache()

        # ----- select best trial -----
        best_trial = max(trials, key=lambda z: z['best_ci'])
        best_hp = best_trial['hp']
        print("\nBest Hyperparameters:")
        print(best_hp)
        print("Best Validation CI (search):", best_trial['best_ci'])

        # ----- save plots per trial -----
        out_date = current_date
        for tr in trials:
            cfg = f"trial{tr['id']}_layers-{'-'.join(map(str, tr['hp']['layers']))}_drop{tr['hp']['dropout']:.2f}_lr{tr['hp']['lr']:.2e}_wd{tr['hp']['wd']:.2e}_l1{tr['hp']['l1']:.2e}"
            epochs = range(1, len(tr['hist_train_ci'])+1)
            plt.figure()
            plt.plot(epochs, tr['hist_train_ci'], label='Train CI')
            plt.plot(epochs, tr['hist_val_ci'], label='Val CI')
            plt.xlabel('Epoch'); plt.ylabel('Concordance Index'); plt.legend(); plt.grid(True, alpha=0.3)
            plt.title(cfg)
            plt.ylim(0.4, 1.0)
            plot_path = os.path.join(output_dir, f"{out_date}_ci_{cfg}.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight'); plt.close()
            print(f"Saved CI plot to {plot_path}")

        # ----- save results table -----
        results = []
        for tr in trials:
            row = {'trial_id': tr['id'], 'val_ci': tr['best_ci'], 'epochs_trained': len(tr['hist_val_ci']),
                   'alive_final': tr['alive']}
            row.update({ 'layers': '-'.join(map(str, tr['hp']['layers'])),
                         'dropout': tr['hp']['dropout'], 'lr': tr['hp']['lr'],
                         'weight_decay(L2)': tr['hp']['wd'], 'l1_lambda': tr['hp']['l1']})
            results.append(row)
        df = pd.DataFrame(results).sort_values('val_ci', ascending=False)
        csv_path = os.path.join(output_dir, f"{out_date}_deepsurv_randomSH_results.csv")
        df.to_csv(csv_path, index=False)
        print(f"Hyperparameter search results saved to {csv_path}")

        # ----- (#3) retrain top-N longer and (optionally) ensemble -----
        sorted_trials = sorted([tr for tr in trials if tr['best_state'] is not None],
                               key=lambda z: z['best_ci'], reverse=True)[:RETRAIN_TOP_N]

        final_states, final_hps, final_val_cis = [], [], []
        for tr in sorted_trials:
            m = DeepSurvMLP(X_train.shape[1], tr['hp']['layers'], dropout=tr['hp']['dropout']).to(device)
            m.load_state_dict(tr['best_state'])
            opt = optim.AdamW(m.parameters(), lr=tr['hp']['lr'], weight_decay=tr['hp']['wd'])
            best_state, best_ci_local, no_improve = copy.deepcopy(tr['best_state']), tr['best_ci'], 0
            for _ in range(MAX_FINAL_EPOCHS):
                train_one_epoch(m, opt, train_loader, device, l1_lambda=tr['hp']['l1'])
                val_ci = evaluate_ci(m, valid_loader, device)
                if val_ci > best_ci_local + 1e-4:
                    best_ci_local, best_state, no_improve = val_ci, copy.deepcopy(m.state_dict()), 0
                else:
                    no_improve += 1
                    if no_improve >= FINAL_PATIENCE: break
            final_states.append(best_state)
            final_hps.append(tr['hp'])
            final_val_cis.append(best_ci_local)
            print(f"[Retrain] Trial {tr['id']} final Val CI: {best_ci_local:.4f}")

        # choose single best after retrain
        best_idx = int(np.argmax(final_val_cis))
        single_best_state = final_states[best_idx]
        single_best_hp = final_hps[best_idx]
        print(f"Best (retrained) single model Val CI: {final_val_cis[best_idx]:.4f}")

        # ensemble on validation
        ens_models = []
        for st, hp in zip(final_states, final_hps):
            mm = DeepSurvMLP(X_train.shape[1], hp['layers'], dropout=hp['dropout']).to(device)
            mm.load_state_dict(st); ens_models.append(mm)
        ens_val_ci = evaluate_ci_ensemble(ens_models, valid_loader, device)
        print(f"Ensemble Val CI: {ens_val_ci:.4f}")

        # ----- save best model (single best after retrain) -----
        best_model_path = os.path.join(output_dir, f"{out_date}_best_deepsurv_model.pth")
        torch.save(single_best_state, best_model_path)
        print(f"Best model saved to {best_model_path}")

        # ----- test evaluation -----
        test_df = pd.read_csv("/content/drive/MyDrive/affyTest.csv")
        if 'Adjuvant Chemo' in test_df.columns:
            test_df['Adjuvant Chemo'] = test_df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})
        for col in binary_columns:
            if col in test_df.columns: test_df[col] = test_df[col].astype(int)

        X_test_full = test_df[feature_cols_all].values.astype(np.float32)
        X_test = scaler.transform(X_test_full[:, topk_idx]).astype(np.float32)

        y_test_time = test_df['OS_MONTHS'].values.astype(np.float32)
        y_test_event = test_df['OS_STATUS'].values.astype(np.float32)

        test_loader = DataLoader(SurvivalDataset(X_test, y_test_time, y_test_event),
                                 batch_size=batch_size, shuffle=False)

        # rebuild & load best single model
        final_model = DeepSurvMLP(X_train.shape[1], single_best_hp['layers'], dropout=single_best_hp['dropout']).to(device)
        final_model.load_state_dict(torch.load(best_model_path, map_location=device))
        test_ci = evaluate_ci(final_model, test_loader, device)
        print(f"Test CI (single best): {test_ci:.4f}")

        # ensemble on test
        test_ens_ci = evaluate_ci_ensemble(ens_models, test_loader, device)
        print(f"Test CI (ensemble): {test_ens_ci:.4f}")

        sys.stdout.flush()

    sys.stdout = original_stdout
    print("Training completed. Check your log file at:", log_path)

if __name__ == "__main__":
    main()

/tmp/ipython-input-3094300462.py:135: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Adjuvant Chemo'] = df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})



=== Rung 1/4 → target 8 epochs ===
[Trial 00 | Rung 1/4 | Epoch   1] Train CI 0.5858 | Val CI 0.6096 | Best -inf
[Trial 00 | Rung 1/4 | Epoch   2] Train CI 0.6294 | Val CI 0.6075 | Best 0.6096
[Trial 00 | Rung 1/4 | Epoch   3] Train CI 0.6602 | Val CI 0.6137 | Best 0.6096
[Trial 00 | Rung 1/4 | Epoch   4] Train CI 0.6813 | Val CI 0.6159 | Best 0.6137
[Trial 00 | Rung 1/4 | Epoch   5] Train CI 0.6955 | Val CI 0.6167 | Best 0.6159
[Trial 00 | Rung 1/4 | Epoch   6] Train CI 0.7119 | Val CI 0.6175 | Best 0.6167
[Trial 00 | Rung 1/4 | Epoch   7] Train CI 0.7175 | Val CI 0.6200 | Best 0.6175
[Trial 00 | Rung 1/4 | Epoch   8] Train CI 0.7266 | Val CI 0.6205 | Best 0.6200
[Trial 01 | Rung 1/4 | Epoch   1] Train CI 0.6126 | Val CI 0.5770 | Best -inf
[Trial 01 | Rung 1/4 | Epoch   2] Train CI 0.6441 | Val CI 0.6077 | Best 0.5770
[Trial 01 | Rung 1/4 | Epoch   3] Train CI 0.6570 | Val CI 0.6114 | Best 0.6077
[Trial 01 | Rung 1/4 | Epoch   4] Train CI 0.6595 | Val CI 0.6169 | Best 0.6114
[Trial 0

/tmp/ipython-input-3094300462.py:322: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['Adjuvant Chemo'] = test_df['Adjuvant Chemo'].replace({'OBS':0,'ACT':1})


Test CI (single best): 0.6292
Test CI (ensemble): 0.6227
Training completed. Check your log file at: /content/drive/MyDrive/deepsurv_8-28-25_training_log.txt
